In [13]:
#Program scrapping FAKE and REAL news data from POLITIFACT.com and use as Dataset. 

In [14]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [15]:

authors = []
dates = []
statements = []
sources = []
targets = []

In [16]:
def scrape_website(page_number):
    page_num = str(page_number)
    URL = 'https://www.politifact.com/factchecks/list/?page='+page_num
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.text, 'html.parser')

    statement_footer = soup.find_all('footer', attrs={'class':'m-statement__footer'})
    statement_quote = soup.find_all('div', attrs= {'class': 'm-statement__quote'})
    statement_meta = soup.find_all('div', attrs={'class': 'm-statement__meta'}) 
    target = soup.find_all('div', attrs={'class':'m-statement__meter'})

    for i in statement_footer:
        link1 = i.text.strip()
        name_and_date = link1.split()
        first_name = name_and_date[1]
        last_name = name_and_date[2]
        full_name = first_name + ' '+ last_name
        month = name_and_date[4]
        day = name_and_date[5]
        year = name_and_date[6]
        date = month+' '+day+' '+year
        dates.append(date)
        authors.append(full_name)

    for i in statement_quote:
        link2 = i.find_all('a')
        statement_text = link2[0].text.strip()
        statements.append(statement_text)

    for i in statement_meta:
        link3 = i.find_all('a')
        source_text = link3[0].text.strip()
        sources.append(source_text)

    for i in target: 
        link4 = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
        targets.append(link4)

In [17]:
n = 150
for i in range(1,n):
    scrape_website(i)

In [18]:
data = pd.DataFrame(columns=['author', 'statement', 'source', 'date', 'target'])
data['author'] = authors
data['statement'] = statements
data['source'] = sources
data['date'] = dates
data['target'] = targets

In [7]:
data

,author,statement,source,date,target
0,Sofia Ahmed,“The population in Gaza has increased by 2.02%...,Instagram posts,"December 6, 2024",false
1,Madison Czopek,"Elon Musk wrote on X that “without CEOs, the w...",Threads posts,"December 6, 2024",false
2,Jeff Cercone,"""Vaccine researcher Peter Hotez says multiple ...",Instagram posts,"December 6, 2024",false
3,Sara Swann,Fox News commentator Dr. Kelly Powers died fro...,Facebook posts,"December 6, 2024",false
4,Loreben Tuquero,“Trump was impeached for trying to investigate...,Instagram posts,"December 6, 2024",false
...,...,...,...,...,...
115,Ciara O'Rourke,"Says Elon Musk said, “I am releasing $1,000 Te...",Facebook posts,"November 12, 2024",false
116,Maria Briceño,“Elon Musk acepta pagar 3.000 millones de dóla...,Instagram posts,"November 12, 2024",false
117,Samantha Putterman,"""BREAKING: Zillow CEO just said on CNBC Trump'...",Instagram posts,"November 12, 2024",false
118,Caleb McCullough,“Elon Musk is taking sustainable technology to...,Facebook posts,"November 12, 2024",false


In [19]:
def getBinaryNumTarget(text):
    if text == 'true':
        return '1'
    else:
        return '0'

In [20]:
def getBinaryTarget(text):
    if text == 'true':
        return 'REAL'
    else:
        return 'FAKE'

In [21]:
data['BinaryTarget']= data['target'].apply(getBinaryTarget)
data['BinaryNumTarget'] = data['target'].apply(getBinaryNumTarget)

In [11]:
data

,author,statement,source,date,target,BinaryTarget,BinaryNumTarget
0,Sofia Ahmed,“The population in Gaza has increased by 2.02%...,Instagram posts,"December 6, 2024",false,FAKE,0
1,Madison Czopek,"Elon Musk wrote on X that “without CEOs, the w...",Threads posts,"December 6, 2024",false,FAKE,0
2,Jeff Cercone,"""Vaccine researcher Peter Hotez says multiple ...",Instagram posts,"December 6, 2024",false,FAKE,0
3,Sara Swann,Fox News commentator Dr. Kelly Powers died fro...,Facebook posts,"December 6, 2024",false,FAKE,0
4,Loreben Tuquero,“Trump was impeached for trying to investigate...,Instagram posts,"December 6, 2024",false,FAKE,0
...,...,...,...,...,...,...,...
115,Ciara O'Rourke,"Says Elon Musk said, “I am releasing $1,000 Te...",Facebook posts,"November 12, 2024",false,FAKE,0
116,Maria Briceño,“Elon Musk acepta pagar 3.000 millones de dóla...,Instagram posts,"November 12, 2024",false,FAKE,0
117,Samantha Putterman,"""BREAKING: Zillow CEO just said on CNBC Trump'...",Instagram posts,"November 12, 2024",false,FAKE,0
118,Caleb McCullough,“Elon Musk is taking sustainable technology to...,Facebook posts,"November 12, 2024",false,FAKE,0


In [22]:
data.to_csv('FactChecker_Dataset.csv')